In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import csv
import warnings

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from konlpy.tag import Okt
import time

class VectorSpaceModel:
    def __init__(self, documents):
        self.documents = documents
        self.vectorizer = TfidfVectorizer()
        self.vectorized_documents = self.vectorizer.fit_transform([doc['article'] for doc in documents])

    def search(self, query):
        query_vector = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vector, self.vectorized_documents)
        ranked_indices = similarities.argsort()[0][::-1]
        return ranked_indices

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', '', text)
    return ' '.join(word_tokenize(text))

def extract_keywords(article_text, num_clusters=5, num_keywords=5):
    okt = Okt()
    preprocessed_text = preprocess_text(article_text)
    words = okt.pos(preprocessed_text, stem=True)
    nouns = [word for word, pos in words if pos == 'Noun' and len(word) > 1]  # 명사이면서 길이가 1보다 긴 것만 선택
    preprocessed_text = ' '.join(nouns)

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([preprocessed_text])

    num_samples = X.shape[0]
    if num_samples < num_clusters:
        num_clusters = num_samples

    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X)

    cluster_centers = kmeans.cluster_centers_
    features = vectorizer.get_feature_names_out()
    top_keywords = []
    for cluster_center in cluster_centers:
        top_keyword_indices = cluster_center.argsort()[-num_keywords:][::-1]
        keywords = ['#' + features[int(i)] for i in top_keyword_indices]
        top_keywords.append(keywords)
    return ' '.join([' '.join(keywords) for keywords in top_keywords])

def save_articles_to_csv(documents, output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'Article'])
        for doc in documents:
            writer.writerow([doc['title'], doc['article']])

documents = []
with open('../datasets/Korea_DB_0413.csv', mode='r', encoding='cp949') as file:
    reader = csv.DictReader(file)

    for row in reader:
        title = row['title']
        article = row['article']
        date = row['date']
        documents.append({'title': title, 'date': date, 'article': article})

vector_space_model = VectorSpaceModel(documents)

query = input("Enter your query: ")

warnings.filterwarnings("ignore", category=FutureWarning)

start = time.time()
ranked_indices = vector_space_model.search(query)
showing_article = []
print("[[ Search results ]]")
print("Below are the articles relevant with the keyword '{}':".format(query))
for i in range(0,100): ######검색결과로 나오는 기사 수는 여기에서 조정하시면 됩니당
    idx=ranked_indices[i]
    showing_article.append(idx)
    print("Title: ", documents[idx]['title'],documents[idx]['date'])
    keywords = extract_keywords(documents[idx]['article'])
    print(keywords, '\n')
end = time.time()
print(f'###########{end-start}##############')
selected_title = input("Enter the title you want to read: ")
print()

for doc in documents:
    if doc['title'] == selected_title:
        print("Title:", doc['title'], '/n')
        print("Article:\n", doc['article'], "/n/n")
        break
else:
    print("Selected title not found in the search results.")

    print("Title:", documents[idx]['title'], '\n')

Enter your query:  가능한 빠르게 처리해주세요


[[ Search results ]]
Below are the articles relevant with the keyword '가능한 빠르게 처리해주세요':
Title:  국힘 공관위원장, 尹·韓 만남에 “굿 뉴스…전체 구도에 굉장히 긍정적” 2024/01/24 11:10
#위원장 #공천 #지역 #전략 #관위 

Title:  ‘흉기 피습’ 이재명 “1월 재판 출석 어렵다”…재판부 “그러면 끝이 없다” 2024/01/12 13:54
#대표 #재판 #진행 #흉기 #출석 

Title:  美 미사일 방어 사령관 “북한 ICBM 미 본토 도달 가능” 2024/03/14 8:16
#북한 #미사일 #미국 #개발 #사령관 

Title:  尹 “2030년엔 인천서 노래 한두 곡 들으면 바로 서울” 2024/03/07 14:46
#인천 #대통령 #서울 #철도 #라며 

Title:  이재명 “친명, 비명 나누는 것은 소명 외면하는 죄악” 2024/02/10 10:56
#대표 #이재명 #국민 #소명 #라며 

Title:  서울 서초·부산 등 76개 자치구 대형마트 일요일 연다 2024/04/02 15:22
#점검 #서비스 #게임 #계획 #조치 

Title:  한 총리 “올해 수출 7000억달러·해외수주 570억달러 이룩할 것” 2024/03/06 10:57
#총리 #산업 #정부 #경제 #지원 

Title:  신장식 변호사, 조국신당 ‘인재1호’ 입당…음주운전 전과엔 “죄송하다” 2024/02/25 12:21
#조국 #정권 #위해 #때문 #정치 

Title:  한동훈, 이재명과 TV토론 수락…총선 전 대담 성사할까 2024/02/29 20:23
#대표 #위원장 #토론 #민주당 #국민 

Title:  국방부 “우리 군 정찰위성 2호기, 4월 초 발사 가능성” 2024/03/26 11:29
#호기 #발사 #정찰위성 #우리 #탑재 

Title:  한 총리, 제주 한라병원 방문…“남은 의료진 소진 막는 데 최선” 2024/04/03 14:57
#병원 #지역 #의료 #한라 

KeyboardInterrupt: Interrupted by user

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import csv
import warnings

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from konlpy.tag import Okt
from sklearn.decomposition import TruncatedSVD

class VectorSpaceModel:
    def __init__(self, documents, num_components=100):
        self.documents = documents
        self.vectorizer = TfidfVectorizer()
        self.vectorized_documents = self.vectorizer.fit_transform([doc['article'] for doc in documents])
        self.svd = TruncatedSVD(n_components=num_components)
        self.reduced_vectorized_documents = self.svd.fit_transform(self.vectorized_documents)

    def search(self, query):
        query_vector = self.vectorizer.transform([query])
        query_vector_reduced = self.svd.transform(query_vector)
        similarities = cosine_similarity(query_vector_reduced, self.reduced_vectorized_documents)
        ranked_indices = similarities.argsort()[0][::-1]
        return ranked_indices

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', '', text)
    return ' '.join(word_tokenize(text))

def extract_keywords(article_text, num_clusters=5, num_keywords=5):
    okt = Okt()
    preprocessed_text = preprocess_text(article_text)
    words = okt.pos(preprocessed_text, stem=True)
    nouns = [word for word, pos in words if pos == 'Noun' and len(word) > 1]  # 명사이면서 길이가 1보다 긴 것만 선택
    preprocessed_text = ' '.join(nouns)

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([preprocessed_text])

    num_samples = X.shape[0]
    if num_samples < num_clusters:
        num_clusters = num_samples

    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(X)

    cluster_centers = kmeans.cluster_centers_
    features = vectorizer.get_feature_names_out()
    top_keywords = []
    for cluster_center in cluster_centers:
        top_keyword_indices = cluster_center.argsort()[-num_keywords:][::-1]
        keywords = ['#' + features[int(i)] for i in top_keyword_indices]
        top_keywords.append(keywords)
    return ' '.join([' '.join(keywords) for keywords in top_keywords])

def save_articles_to_csv(documents, output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'Article'])
        for doc in documents:
            writer.writerow([doc['title'], doc['article']])

documents = []
with open('../datasets/Korea_DB_0413.csv', mode='r', encoding='cp949') as file:
    reader = csv.DictReader(file)

    for row in reader:
        title = row['title']
        article = row['article']
        date = row['date']
        documents.append({'title': title, 'date': date, 'article': article})

vector_space_model = VectorSpaceModel(documents)

query = input("Enter your query: ")

warnings.filterwarnings("ignore", category=FutureWarning)
start = time.time()
ranked_indices = vector_space_model.search(query)
showing_article = []
print("[[ Search results ]]")
print("Below are the articles relevant with the keyword '{}':".format(query))
for i in range(0,100): ######검색결과로 나오는 기사 수는 여기에서 조정하시면 됩니당
    idx=ranked_indices[i]
    showing_article.append(idx)
    print("Title: ", documents[idx]['title'],documents[idx]['date'])
    keywords = extract_keywords(documents[idx]['article'])
    print(keywords, '\n')
end = time.time()
print(f'###########{end-start}##############')
selected_title = input("Enter the title you want to read: ")
print()

for doc in documents:
    if doc['title'] == selected_title:
        print("Title:", doc['title'], '/n')
        print("Article:\n", doc['article'], "/n/n")
        break
else:
    print("Selected title not found in the search results.")

    print("Title:", documents[idx]['title'], '\n')

Enter your query:  가능한 빠르게 처리해주세요


[[ Search results ]]
Below are the articles relevant with the keyword '가능한 빠르게 처리해주세요':
Title:  어딘지도 모르는데 종류도 제각각…행안부 “대피소 공동활용 추진” 2024/01/12 12:51
#대피소 #재난 #활용 #시설 #종류 

Title:  軍, 신형 위성위치보고장치 도입…특수작전 능력 강화 2024/02/22 15:00
#위성 #위치 #보고 #장치 #통신 

Title:  사과 등 10대 농산물 중심 ‘최대 인력’ 공급…“가격 안정 기대” 2024/04/05 16:55
#인력 #계절 #올해 #수요 #농번기 

Title:  韓 최초 ‘초소형군집위성 1호’ 다음달 쏜다 2024/03/29 10:47
#위성 #발사 #군집 #초소 #카이스트 

Title:  민원·공공서비스 구비서류 없앤다…인감도 ‘디지털’ 전환 2024/01/30 12:24
#서류 #서비스 #발급 #신청 #국민 

Title:  KAI, 차세대 발사체 입찰 불참…“독자 우주모빌리티 집중” 2024/02/21 15:08
#우주 #개발 #사업 #발사체 #기업 

Title:  서울 서초·부산 등 76개 자치구 대형마트 일요일 연다 2024/04/02 15:22
#점검 #서비스 #게임 #계획 #조치 

Title:  3600t급 장보고 3번함 건조 착수…탐지·표적처리 성능 개선 2024/01/31 15:49
#건조 #방사청 #착수 #전력 #사업 

Title:  지역전략사업땐 그린벨트 총량 제한 없이 푼다…1·2등급지도 해제 2024/02/21 15:59
#지역 #해제 #농지 #정부 #계획 

Title:  軍, M61 기관포 등 11개 무기체계 성능 빠르게 개선키로 2024/02/13 16:15
#개선 #사업 #성능 #신속 #효율 

Title:  정부, 고흥 우주발사체 국가산업단지 ‘예타 면제’ 추진 2024/02/14 14:23
#산업 #단지 #국가 #우주발사체 #고흥 

Title:  北 잇단 미사일 도발에 日도쿄

KeyboardInterrupt: Interrupted by user